In [1]:
from aicspylibczi import CziFile
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import matplotlib.animation as animation
import IPython
from IPython.display import HTML

import cv2
import os
import ffmpeg
import time
import pandas as pd
from cellpose import io, models

In [2]:
from skimage.measure import label # version at least 0.22
from skimage.measure import regionprops_table # version at least 0.22
from skimage.filters import threshold_otsu # version at least 0.22
from utils import *

In [3]:
video = get_file('mip', 3)



Loading dicty_factin_pip3-03_MIP.czi with dims [{'X': (0, 474), 'Y': (0, 2048), 'C': (0, 2), 'T': (0, 90)}]


In [4]:
frames, shp = video.read_image(C=0)
frames = scale_img(frames.squeeze())
frames.shape

(90, 2048, 474)

In [5]:
masks = binarize_video(frames)

##### Helper Functions

In [6]:
def bounding_boxes(mask, min_area=300):    
    # deducing centroids, max side length, 
    all_bboxes_coords = list()
    max_num_cells=0
    num_cells_frames = list()
    # region props can calculate a LOT of useful properties
    # might want to look into better cell filtering
    props = regionprops_table(label_image=mask,
                              properties=['bbox','area']
    )
    # each row corresponds to cell id, columns corresponds to properties
    # box returned as min_row, min_col, max_row, max_col
    df = pd.DataFrame(props)
    df = df[df['area'] >= min_area]
    cell_areas = df['area'].values
    df= df.drop(columns = ['area'])
    bboxes, num_cells = list(df.itertuples(index=False, name=None)), df.shape[0]
    return bboxes, num_cells, cell_areas

In [27]:
def track_cells(cell_id, frames, masks, padding=0):
    N = frames.shape[0]
    if(type(masks) == int):
        # if not provided, calculated
        masks = binarize_video(frames)
        print("Computed binary masks")

    data = {"patches" : [], 'boxes' : [], "masks" : []}
    areas = []
    num_cells_frames = []
    
    def get_corresponding_cell_box(boxes, num, areas, frame_idx=0):
        ## TO DO : Ensure that index of boxes stays consistent even as boxes appear and dissapear
        if(frame_idx == 0):
            return(cell_id)

        assert len(data['boxes']) >= frame_idx, f"Missing frame history at {frame_idx}"
        prev_box, prev_num, prev_area = data['boxes'][-1], num_cells_frames[-1], areas[-1]
        scores = []
        for i in range(len(boxes)):
            delta_area = abs(prev_area - areas[i])
            delta_x = abs(prev_box[0] - boxes[i][0])
            delta_y = abs(prev_box[1] - boxes[i][1])

            score = 50*(delta_x + delta_y) + delta_area / 10
            scores.append(score)

        index = np.argmin(scores)
        if(index != cell_id):
            print(f"Choosing {index} instead of cell id {cell_id} at frame {frame_idx}")
            print(f"Areas: {areas}\n boxes: {boxes}")
            print(f"Previous box and area: {prev_box}\n {prev_area}")
            print(f"Cell id area changed from {prev_area} to areas[cell_id]. All areas are {areas}")
            print("\n\n\n")
        return(index)
    
    h_max, w_max = 0, 0
    skip_frames = []
    for frame_idx in range(N):
        boxes, num_cells, area = bounding_boxes(masks[frame_idx]) # all on frame
        index_of_cell = get_corresponding_cell_box(boxes, num_cells, area, frame_idx) # right now assume same index throughout entire video
        
        areas.append(area[index_of_cell])
        num_cells_frames.append(num_cells)
        
        bbox = boxes[index_of_cell] # cell id is meant to track a SPECIFIC cell, ensure maintain integrity of cell over time
        
        min_row, min_col, max_row, max_col = bbox

        # adjust boxes based on padding, 
        #TODO might expand boxes to homogenize sizes; 
        min_row -= padding
        min_col -= padding
        max_row += padding
        max_col += padding

        data['boxes'].append((min_row, min_col, max_row, max_col))
        
        patch = frames[frame_idx, min_row:max_row, min_col:max_col]
        mask_patch = masks[frame_idx, min_row:max_row, min_col:max_col]
        data['patches'].append(patch)
        data['masks'].append(mask_patch)

    return(data)

In [28]:
data = track_cells(2, frames, masks, padding=0)

Choosing 1 instead of cell id 2 at frame 3
Areas: [ 9175. 10540.   408.  3777.  4777.  5105.]
 boxes: [(468, 33, 619, 151), (904, 229, 1049, 335), (1021, 75, 1060, 99), (1366, 245, 1444, 324), (1641, 71, 1750, 150), (1874, 43, 1988, 118)]
Previous box and area: (910, 229, 1053, 333)
 5105.0
Cell id area changed from 5105.0 to areas[cell_id]. All areas are [ 9175. 10540.   408.  3777.  4777.  5105.]




Choosing 3 instead of cell id 2 at frame 7
Areas: [ 6197.   427.   648. 11092.   425.  3917.  4611.  4404.  3979.]
 boxes: [(465, 84, 610, 155), (490, 36, 514, 61), (526, 9, 564, 37), (897, 219, 1049, 336), (1021, 75, 1060, 100), (1344, 262, 1438, 325), (1573, 82, 1699, 161), (1905, 43, 1987, 132), (1991, 169, 2048, 262)]
Previous box and area: (901, 218, 1045, 335)
 3979.0
Cell id area changed from 3979.0 to areas[cell_id]. All areas are [ 6197.   427.   648. 11092.   425.  3917.  4611.  4404.  3979.]




Choosing 1 instead of cell id 2 at frame 11
Areas: [ 6564. 10984.   454.  3901.  3

In [29]:
def animate_frames(frames):
    fig = plt.figure()
    im = plt.imshow(frames[0])
    
    plt.close() # this is required to not display the generated image
    
    def init():
        im.set_data(frames[0])
    
    def animate(i):
        im.set_data(frames[i])
        return im
    
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(frames),
                                   interval=200)
    
    HTML(anim.to_html5_video())
    plt.show()
    return(anim)

In [30]:
data['boxes']

[(900, 228, 1042, 332),
 (904, 228, 1048, 333),
 (910, 229, 1053, 333),
 (904, 229, 1049, 335),
 (912, 227, 1050, 335),
 (907, 225, 1047, 334),
 (901, 218, 1045, 335),
 (897, 219, 1049, 336),
 (900, 219, 1052, 337),
 (900, 218, 1056, 336),
 (902, 218, 1055, 334),
 (904, 215, 1052, 334),
 (907, 217, 1050, 334),
 (911, 217, 1047, 333),
 (905, 211, 1051, 333),
 (908, 217, 1055, 334),
 (905, 219, 1052, 334),
 (907, 223, 1043, 334),
 (909, 221, 1044, 335),
 (913, 222, 1039, 336),
 (911, 224, 1039, 335),
 (916, 222, 1040, 335),
 (919, 219, 1041, 336),
 (917, 221, 1041, 334),
 (923, 222, 1039, 334),
 (924, 220, 1039, 334),
 (924, 224, 1039, 333),
 (924, 223, 1040, 332),
 (920, 224, 1041, 333),
 (921, 225, 1042, 332),
 (924, 225, 1040, 334),
 (920, 225, 1038, 333),
 (918, 223, 1037, 333),
 (920, 222, 1037, 335),
 (919, 222, 1037, 332),
 (921, 224, 1038, 332),
 (921, 224, 1036, 332),
 (921, 223, 1036, 333),
 (921, 222, 1036, 333),
 (919, 220, 1036, 332),
 (920, 223, 1035, 333),
 (920, 223, 1038

In [31]:
anim = animate_frames(data['patches'])
HTML(anim.to_html5_video())

In [22]:
masks == "asd"

/tmp/ipykernel_11704/2247525644.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  masks == "asd"


False